In [ ]:
import sys
from math import fabs
import rasterio
from rasterio.transform import Affine
from pyproj import Transformer, Geod

In [2]:
def pixel_and_area(filepath):
    with rasterio.open(filepath) as src:
        transform: Affine = src.transform
        width = src.width
        height = src.height
        crs = src.crs

        # affine: [a, b, c, d, e, f] where:
        # a = pixel width (x scale)
        # e = pixel height (negative if origin top-left)
        a = transform.a
        e = transform.e

        pixel_size_x_crs = fabs(a)        # units = CRS units per pixel (e.g., meters or degrees)
        pixel_size_y_crs = fabs(e)        # units = CRS units per pixel

        # center coordinates (in raster CRS)
        center_col = width / 2.0
        center_row = height / 2.0
        center_x, center_y = rasterio.transform.xy(transform, center_row, center_col, offset='center')

        print(f"File: {filepath}")
        print(f"Raster size: {width} x {height} (cols x rows)")
        print(f"CRS: {crs}")
        print(f"Affine transform: {transform}")
        print()
        print(f"Pixel size in CRS units:  x = {pixel_size_x_crs:.6f}, y = {pixel_size_y_crs:.6f} (CRS units / pixel)")

        # If CRS units are degrees (EPSG:4326 or geographic), compute meters using geodetic measurement
        if crs is not None and crs.is_geographic:
            print("CRS is geographic (degrees). Estimating pixel size in meters at raster center...")

            # define two points one pixel apart horizontally and vertically, transform to lon/lat
            # center_x, center_y are in lon/lat already for geographic CRS
            lon0, lat0 = center_x, center_y

            # horizontal neighbor: move by one pixel in x (CRS degrees)
            lon1 = lon0 + (a)  # a may be positive or negative, but using as-is works
            lat1 = lat0

            # vertical neighbor: move by one pixel in y (CRS degrees)
            # note: e is usually negative for north-up rasters; add e to y
            lon2 = lon0
            lat2 = lat0 + (e)

            geod = Geod(ellps='WGS84')
            # distance measurements (meters)
            _, _, dx_m = geod.inv(lon0, lat0, lon1, lat1)
            _, _, dy_m = geod.inv(lon0, lat0, lon2, lat2)

            print(f"Estimated pixel size in meters: x ≈ {abs(dx_m):.3f} m, y ≈ {abs(dy_m):.3f} m")

            # area estimate (approx): number of pixels * average pixel area (dx*dy)
            approx_area_m2 = width * height * abs(dx_m) * abs(dy_m)
            print(f"Approximate coverage area: {approx_area_m2:,.0f} m² ({approx_area_m2/1e6:.3f} km²)")

        else:
            # assumed projected CRS with units in meters (common: EPSG:3857, UTM, etc.)
            print("CRS appears projected (units likely meters).")
            print(f"Pixel size in meters (assuming CRS units = meters): x = {pixel_size_x_crs:.6f} m, y = {pixel_size_y_crs:.6f} m")
            pixel_area_m2 = pixel_size_x_crs * pixel_size_y_crs
            total_area_m2 = pixel_area_m2 * width * height
            print(f"Pixel area = {pixel_area_m2:.6f} m²")
            print(f"Total coverage area = {total_area_m2:,.0f} m² ({total_area_m2/1e6:.3f} km²)")

        # return values for programmatic use
        return {
            "width_px": width,
            "height_px": height,
            "pixel_x_crs": pixel_size_x_crs,
            "pixel_y_crs": pixel_size_y_crs,
            "crs": crs.to_string() if crs is not None else None,
            "center": (center_x, center_y),
        }




def main():
    if len(sys.argv) < 2:
        print("Usage: python get_pixel_size.py /path/to/orthomosaic.tif")
        sys.exit(1)

    file_path = "D:/2_Analytics/9_LULC_classification/demo_ortho/cropped_ortho.tif"
    pixel_and_area(file_path)

if __name__ == "__main__":
    main()


File: D:/2_Analytics/9_LULC_classification/demo_ortho/cropped_ortho.tif
Raster size: 59328 x 60021 (cols x rows)
CRS: EPSG:32644
Affine transform: | 0.03, 0.00, 656270.31|
| 0.00,-0.03, 2485024.69|
| 0.00, 0.00, 1.00|

Pixel size in CRS units:  x = 0.030000, y = 0.030000 (CRS units / pixel)
CRS appears projected (units likely meters).
Pixel size in meters (assuming CRS units = meters): x = 0.030000 m, y = 0.030000 m
Pixel area = 0.000900 m²
Total coverage area = 3,204,833 m² (3.205 km²)


In [10]:
# Downsample single-band (or multi-band) GeoTIFF safely, and optionally expand single-band -> 3-band
import os
import rasterio
from rasterio.warp import reproject, Resampling as WarpResampling
from rasterio.transform import Affine
from rasterio.enums import Resampling

def downsample_safe(input_tiff: str, output_tiff: str, target_pixel: float,
                    resampling=Resampling.average, expand_to_3bands=False):
    """
    Downsample input_tiff -> output_tiff with pixel size target_pixel (CRS units).
    Works for 1-band and multi-band rasters. If expand_to_3bands=True and source has 1 band,
    output will have 3 identical bands (useful if downstream expects RGB).
    Uses a minimal profile (no tiling/compression) for maximum compatibility.
    """
    # remove existing output
    if os.path.exists(output_tiff):
        print("Removing existing output:", output_tiff)
        os.remove(output_tiff)

    with rasterio.open(input_tiff) as src:
        src_transform = src.transform
        src_crs = src.crs
        src_width = src.width
        src_height = src.height
        src_count = src.count
        src_dtype = src.dtypes[0]
        src_nodata = src.nodata

        src_px_x = abs(src_transform.a)
        src_px_y = abs(src_transform.e)

        print(f"Input: {input_tiff}")
        print(f"Raster size: {src_width} x {src_height}, bands: {src_count}")
        print(f"Current pixel size: x={src_px_x}, y={src_px_y}")

        scale_x = target_pixel / src_px_x
        scale_y = target_pixel / src_px_y

        dst_width = int(round(src_width / scale_x))
        dst_height = int(round(src_height / scale_y))

        dst_transform = Affine(src_transform.a * scale_x,
                               src_transform.b,
                               src_transform.c,
                               src_transform.d,
                               src_transform.e * scale_y,
                               src_transform.f)

        # decide output band count
        if src_count == 1 and expand_to_3bands:
            dst_count = 3
        else:
            dst_count = src_count

        print("Target pixel size:", target_pixel)
        print("Output size:", dst_width, "x", dst_height)
        print("Output bands:", dst_count)

        profile = {
            "driver": "GTiff",
            "height": dst_height,
            "width": dst_width,
            "count": dst_count,
            "dtype": src_dtype,
            "crs": src_crs,
            "transform": dst_transform,
        }
        if src_nodata is not None:
            profile["nodata"] = src_nodata

        # Create destination file with minimal profile (no tiling/compression)
        with rasterio.open(output_tiff, "w", **profile) as dst:
            # For each output band, reproject appropriate source band (or the single source band duplicated)
            for out_band in range(1, dst_count + 1):
                # choose source band index to use for this output band
                # if src has multiple bands use the same band index (1->1, 2->2, 3->3) up to src_count
                # if expand_to_3bands and src_count==1, always use source band 1
                if src_count >= out_band:
                    src_band_idx = out_band
                else:
                    src_band_idx = 1  # duplicate band 1 for remaining output bands

                print(f"Resampling output band {out_band} from source band {src_band_idx} ...", end="", flush=True)

                reproject(
                    source=rasterio.band(src, src_band_idx),
                    destination=rasterio.band(dst, out_band),
                    src_transform=src_transform,
                    src_crs=src_crs,
                    dst_transform=dst_transform,
                    dst_crs=src_crs,
                    resampling=WarpResampling.average if resampling == Resampling.average else WarpResampling.nearest
                )
                print(" done.")

    # quick check
    with rasterio.open(output_tiff) as r:
        print("Saved:", output_tiff)
        print("W x H:", r.width, r.height)
        print("Bands:", r.count)
        print("Pixel size:", abs(r.transform.a), abs(r.transform.e))
        print("dtype:", r.dtypes)
        print("nodata:", r.nodata)

# === Usage examples ===
inp = r"D:/2_Analytics/9_LULC_classification/demo_ortho/cropped_ortho.tif"
outp_single = r"D:/2_Analytics/9_LULC_classification/demo_ortho/cropped_ortho_3m_singleband.tif"
outp_rgb = r"D:/2_Analytics/9_LULC_classification/demo_ortho/cropped_ortho_3m_rgb.tif"

# If your orthomosaic truly has 1 band and your classifier expects RGB, set expand_to_3bands=True
downsample_safe(inp, outp_single, target_pixel=3.0, resampling=Resampling.average, expand_to_3bands=False)

# If you need a 3-band version (duplicate single band into 3 channels)
downsample_safe(inp, outp_rgb, target_pixel=3.0, resampling=Resampling.average, expand_to_3bands=True)


Removing existing output: D:/2_Analytics/9_LULC_classification/demo_ortho/cropped_ortho_3m_singleband.tif
Input: D:/2_Analytics/9_LULC_classification/demo_ortho/cropped_ortho.tif
Raster size: 59328 x 60021, bands: 1
Current pixel size: x=0.02999999999999945, y=0.02999999999999814
Target pixel size: 3.0
Output size: 593 x 600
Output bands: 1
Resampling output band 1 from source band 1 ... done.
Saved: D:/2_Analytics/9_LULC_classification/demo_ortho/cropped_ortho_3m_singleband.tif
W x H: 593 600
Bands: 1
Pixel size: 3.0 3.0
dtype: ('uint8',)
nodata: None
Input: D:/2_Analytics/9_LULC_classification/demo_ortho/cropped_ortho.tif
Raster size: 59328 x 60021, bands: 1
Current pixel size: x=0.02999999999999945, y=0.02999999999999814
Target pixel size: 3.0
Output size: 593 x 600
Output bands: 3
Resampling output band 1 from source band 1 ... done.
Resampling output band 2 from source band 1 ... done.
Resampling output band 3 from source band 1 ... done.
Saved: D:/2_Analytics/9_LULC_classificatio